In [1]:
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()

True

In [8]:
from langchain.agents import tool

@tool
def add(a, b) -> int:
    """Adds a and b."""
    return int(a) + int(b)

@tool
def subtract(a, b) -> int:
    """subtract  b from  a."""
    return int(a) -int(b)


@tool
def multiply(a, b) -> int:
    """Multiplies a and b."""
    return int(a) * int(b)

In [9]:
multiply

StructuredTool(name='multiply', description='Multiplies a and b.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x000002169D78FF60>)

In [13]:
tools = [ multiply,subtract]

In [ ]:
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")


@tool("search-tool", args_schema=SearchInput)
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

search

StructuredTool(name='search-tool', description='Look up things online.', args_schema=<class '__main__.SearchInput'>, return_direct=True, func=<function search at 0x000002169D78F600>)

In [10]:
from langchain.tools import  StructuredTool

def sivamultiply(a, b) -> int:
    """Multiplies a and b."""
    return int(a) * int(b)

class MultiplyInput(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

multiplytool=StructuredTool.from_function(
                        func=sivamultiply,
                        name="sivaprasadmultiply",
                        description="Multiplies two numbers",
                        args_schema=MultiplyInput,
                    )
multiplytool

StructuredTool(name='sivaprasadmultiply', description='Multiplies two numbers', args_schema=<class '__main__.MultiplyInput'>, func=<function sivamultiply at 0x00000216CAB2B600>)

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

taviltysearch = TavilySearchResults(max_results=5)
taviltysearch.invoke("Tell me about Sivaprasad Valluru")




[{'url': 'https://www.lwww.in/home/about-us',
  'content': 'Home About Self Paced Course Instructor Led Course Contact SivaPrasad Valluru is a instructor with 20 years of experince in IT . Now, he delivers corporate trainings. He got certified as a instructor from Mulesoft and Pivotal and has delivered many certified trainings. He spends a lot of time in understanding the frameworks and explores the internal workings. Since he knows most of the internals, he delivers training with confidence. While delivering trainings, he never uses high level jargons and presentations. Once you attend his training, you will be able to explain any concepts clearly and will answer any questions logically in interviews Advance Courses Want to talk to us reagrding any course or live training or corporate training requirement?'},
 {'url': 'https://www.linkedin.com/in/sivaprasad-valluru-294b0528',
  'content': 'Siva is a skilled Web Dynpro and Java developer, and doubled up as a NWDI administrator during t

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI()
tools = [taviltysearch,add,subtract,multiply]

llm_with_tools=llm.bind_tools(tools)

llm_with_tools.invoke(" who is Sachin tendulkar ")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bZV4GhUVRe2A33tpxeEb8Ubj', 'function': {'arguments': '{"query":"Sachin Tendulkar"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 161, 'total_tokens': 186, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-35724fc7-4901-4b14-985d-35c9b199cd8c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Sachin Tendulkar'}, 'id': 'call_bZV4GhUVRe2A33tpxeEb8Ubj', 'type': 'tool_call'}], usage_metadata={'input_tokens': 161, 'output_tokens': 25, 'total_tokens': 186, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_to

In [18]:
from typing import Union, List
from langchain.tools import Tool

def find_tool_by_name(tools: List[Tool], tool_name: str) -> Tool:
    for tool in tools:
        if tool.name == tool_name:
            return tool
    raise ValueError(f"Tool wtih name {tool_name} not found")

find_tool_by_name(tools, "add")

StructuredTool(name='add', description='Adds a and b.', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x00000216CAB2BA60>)

In [19]:
from langchain.tools.render import render_text_description
render_text_description(tools)

'tavily_search_results_json - A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.\nadd(a, b) -> int - Adds a and b.\nsubtract(a, b) -> int - subtract  b from  a.\nmultiply(a, b) -> int - Multiplies a and b.'

In [21]:
from langchain.prompts import PromptTemplate
from langchain.tools.render import render_text_description

tools = [taviltysearch,add,subtract,multiply]


template = """
    Answer the following questions as best you can. You have access to the following tools:

    {tools}

    You have to use these tools only even if u can compute the response directly.
    Use one tool at a time

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: {input}
    Thought: {agent_scratchpad}
    """

prompt = PromptTemplate.from_template(template=template).partial(tools=render_text_description(tools),
                                                                 tool_names=[tool.name for tool in tools],)


In [ ]:
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str

llm= ChatOpenAI(stop_sequences=["\nObservation","Observation"])
agent = {
     "input": lambda x: x['input'],
     "agent_scratchpad": lambda x: format_log_to_str(x['intermediate_steps'])  
}| prompt | llm | ReActSingleInputOutputParser()

intermediate_steps= []

agentaction=agent.invoke({
    "input": "what is  sum of 6  and 7 multipled by 5",
    "intermediate_steps": []
})
print(agentaction)
#
##
intermediate_steps.append((agentaction,13))


tool='add' tool_input='6, 7' log='I can start by adding 6 and 7 together, then multiply the result by 5 to find the final answer.\n\nAction: add\nAction Input: 6, 7'


In [29]:
agentaction=agent.invoke({
    "input": "what is  sum of 6  and 7 multipled by 5",
    "intermediate_steps": intermediate_steps
})
print(agentaction)
intermediate_steps.append((agentaction,65))
format_log_to_str(intermediate_steps)

tool='multiply' tool_input='13, 5' log='I now know the sum of 6 and 7 is 13, next I will multiply it by 5 to get the final answer.\n\nAction: multiply\nAction Input: 13, 5'


'I can start by adding 6 and 7 together, then multiply the result by 5 to find the final answer.\n\nAction: add\nAction Input: 6, 7\nObservation: 13\nThought: I now know the sum of 6 and 7 is 13, next I will multiply it by 5 to get the final answer.\n\nAction: multiply\nAction Input: 13, 5\nObservation: 65\nThought: '

In [30]:
agentaction=agent.invoke({
    "input": "what is  sum of 6  and 7 multipled by 5",
    "intermediate_steps": intermediate_steps
})
print(agentaction)

return_values={'output': 'The sum of 6 and 7 multiplied by 5 is 65.'} log='Final Answer: The sum of 6 and 7 multiplied by 5 is 65.'


In [68]:
from langchain_core.output_parsers import StrOutputParser
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str

intermediate_steps= []

chain=  {
    "input": lambda x: x['input'],
    "agent_scratchpad": lambda x: format_log_to_str(x['intermediate_steps']),
}| prompt | llm | ReActSingleInputOutputParser()

In [69]:
agentaction=chain.invoke({
    "input": "what is  sum of 6  and 7 multipled by 5",
    "intermediate_steps": intermediate_steps
})
print(agentaction)

tool='add' tool_input='6, 7\n' log='Thought: To solve this problem, we need to perform addition, multiplication, and possibly subtraction.\n    Action: add\n    Action Input: 6, 7\n    '


In [63]:
intermediate_steps.append((agentaction,13))

In [64]:
agentaction=chain.invoke({
    "input": "what is  sum of 6  and 7 multipled by 5",
    "intermediate_steps": intermediate_steps
})
agentaction

AgentAction(tool='multiply', tool_input='13, 5', log='I added 6 and 7, and the result is 13. Now, I need to multiply this sum by 5.\nAction: multiply\nAction Input: 13, 5')

In [65]:
intermediate_steps.append((agentaction,65))

In [66]:
agentaction=chain.invoke({
    "input": "what is  sum of 6  and 7 multipled by 5",
    "intermediate_steps": intermediate_steps
})
agentaction

AgentFinish(return_values={'output': '65'}, log='Final Answer: 65')

In [78]:
from langchain.schema import AgentAction, AgentFinish

agent_step = ""
intermediate_steps=[]

while not isinstance(agent_step, AgentFinish):
  agent_step: Union[AgentAction, AgentFinish] = chain.invoke(
            {
                "input": "Who is Sivaprasad valluru",
                "intermediate_steps": intermediate_steps,
            }
  )
  print(f"{agent_step=}")
  if isinstance(agent_step, AgentAction):
    tool_name = agent_step.tool
    tool_to_use = find_tool_by_name(tools, tool_name)
    tool_input = agent_step.tool_input
    tool_input=tool_input.split(",")
    tool_input=[ input.strip()  for input in tool_input]

    if(len(tool_input)==1):
        tool_input=tool_input[0]

    print(f"{tool_input=}")

    observation = tool_to_use.invoke(tool_input)
    print(f" invoked the tool {tool_name = } on {tool_input=}. Result {observation=}")
    intermediate_steps.append((agent_step, str(observation)))
    print(f"intermediate_steps {intermediate_steps=}")

    if isinstance(agent_step, AgentFinish):
          print(agent_step.return_values)
  

agent_step=AgentAction(tool='tavily_search_results_json', tool_input='Sivaprasad Valluru\n', log='Thought: I should use the `tavily_search_results_json` tool to find information on Sivaprasad Valluru.\n    Action: tavily_search_results_json\n    Action Input: Sivaprasad Valluru\n    ')
tool_input='Sivaprasad Valluru'
 invoked the tool tool_name = 'tavily_search_results_json' on tool_input='Sivaprasad Valluru'. Result observation=[{'url': 'https://www.indiafilings.com/search/sivaprasad-valluru-din-08853173', 'content': 'SIVAPRASAD VALLURU bearing DIN number 08853173 is currently the director of HEAL FORCE INDIA PRIVATE LIMITED, ARTIZAN MEDITEK PRIVATE LIMITED.'}, {'url': 'https://www.lwww.in/home/about-us', 'content': 'Home About Self Paced Course Instructor Led Course Contact SivaPrasad Valluru is a instructor with 20 years of experince in IT . Now, he delivers corporate trainings. He got certified as a instructor from Mulesoft and Pivotal and has delivered many certified trainings. He

In [32]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [35]:
from langchain.agents import create_react_agent,AgentExecutor

llm= ChatOpenAI()

agent=create_react_agent(llm=llm,prompt=prompt,tools=tools)

agentexecutor=AgentExecutor(
    agent=agent, tools=tools
)
response = agentexecutor.invoke({"input": "tell me about Keerthana Jothi ramesh kumar"})
response

{'input': 'tell me about Keerthana Jothi ramesh kumar',
 'output': 'Keerthana Jothi Ramesh Kumar is a female chess player from India with a FIDE ID of 25640283 and an inactive standard rating of 1514.'}

In [38]:
from langchain.agents import  create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate,HumanMessagePromptTemplate,MessagesPlaceholder
from langchain.schema import SystemMessage

prompt = ChatPromptTemplate(
    messages=[
        SystemMessage(content=(
            "You are an AI ."
            "You should not use tools parallelly "
            "You should not make your own conclusions even if it is easy. instead use appropriate tools "
        )),

        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

agent = create_tool_calling_agent(llm=llm, tools=tools,prompt=prompt)


agent_executor = AgentExecutor(
    agent=agent,    verbose=True,    max_iterations=20,    tools=tools,)
response = agent_executor.invoke({"input": "tell me about Dhanush Duraipandian"})
response



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Dhanush Duraipandian'}`


[{'url': 'https://www.tiktok.com/@duraipandian27/video/6820421372964064514', 'content': 'TikTok video from Durai Pandian (@duraipandian27): “”. original sound - Dhanush ..... balu.. - rng Balu❤️.'}, {'url': 'https://www.instagram.com/_d_pandi_/reel/CJ3OrEmjAe2/', 'content': 'Photo by Durai pandian on May 20, 2024.'}, {'url': 'https://coda.io/@kvaluent/pd176-gen-ai/gradebook-2', 'content': '87.14%. Anuvarshini S S. 54.29%. Ayush U. 77.14%. Balaharinath C. 57.14%. Burlagadda Sai Nikhil. 77.14%. Dhanush Duraipandian. 82.86%. Anirudh G. 71.43%. Gouri'}, {'url': 'https://www.linkedin.com/pub/dir/Dhanush/Durai?trk=public_profile_samename-see-all', 'content': '7Results for "Dhanush Durai" ; Planning and Cost control Manager at Kentz. Yemen. Kentz ; Attended GRD SCHOOL OF COMMERCE AND INTERNATIONAL BUSINESS (GRDSCIB).'}, {'url': 'https://in.linkedin.com/in/dhanushree-durai

{'input': 'tell me about Dhanush Duraipandian',
 'output': 'Dhanush Duraipandian is associated with various social media platforms like TikTok and Instagram. It seems there are individuals with this name on different platforms. On TikTok, you can find a video by Durai Pandian with the username duraipandian27. Moreover, a person named Dhanush Duraipandian appears to have studied at the Government College of Technology in Coimbatore and has connections on LinkedIn. Additionally, another individual named Dhanush Duraipandian is mentioned in a document related to academic performance.'}

In [40]:

from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=1000)

print(api_wrapper.run("Sachin Tendulkar"))

Page: Sachin Tendulkar
Summary: Sachin Ramesh Tendulkar ( ; pronounced [sətɕin teːɳɖulkəɾ]; born 24 April 1973) is an Indian former international cricketer who captained the Indian national team. He is widely regarded as one of the greatest cricketers of all time, and is the holder of several world records, including being the all-time highest run-scorer in both ODI and Test cricket, receiving the most player of the match awards in international cricket, and being the only batsman to score 100 international centuries. Tendulkar was a Member of Parliament, Rajya Sabha by presidential nomination from 2012 to 2018. 
Tendulkar took up cricket at the age of eleven, made his Test match debut on 15 November 1989 against Pakistan in Karachi at the age of sixteen, and went on to represent Mumbai domestically and India internationally for over 24 years. In 2002, halfway through his career, Wisden ranked him the second-greatest Test batsman of all time, behind Don Bradman, and the second-greatest

In [41]:
from langchain_community.tools import WikipediaQueryRun
from pydantic import BaseModel, Field
#from langchain_core.pydantic_v1 import BaseModel, Field

class WikiInputs(BaseModel):
    """Inputs to the wikipedia tool."""

    query: str = Field(
        description="query to look up in Wikipedia, should be 3 or less words"
    )

tool = WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)
tool

WikipediaQueryRun(name='wiki-tool', description='look up things in wikipedia', args_schema=<class '__main__.WikiInputs'>, return_direct=True, api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\prodapt\\workspace\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=1000))

In [2]:
from langchain_community.utilities import SerpAPIWrapper

search = SerpAPIWrapper()
print(search.run("tell about SivaPrasad Valluru  in 5 bullet points. I want only info about him "))

['Siva Prasad Valluru. Professional Freelance Trainer in Mulesoft & Generative AI | Expertise in LangChain, Llama Index, LangGraph, CrewAI, OpenAI Swarm, OpenAI ...', "Siva Prasad Valluru's Post. View profile for Siva Prasad Valluru. Siva Prasad Valluru. Professional Freelance Trainer in Mulesoft & Generative ...", 'This paper describes an asymptomatic presentation of a foreign body in the oral mucosa which clinically appeared like a benign connective tissue tumor. Unusual ...', 'The book is divided into six parts or sections: (1) Chronology; (2) Placewise. Index; (3) General Index; (4) Statement of Fasts; (5) Statement ...', 'Traditionally, universities have been governed on a collegiate basis, focusing on developing knowledge both in the students attending and ...', 'REGISTRAR OF NEWSPAPER FOR INDIA. MINISTRY OF INFORMATION & BROADCASTING. GOVERNMENT OF INDIA. Page 2. Page 3. CONTENTS. Volume –II. Page No.', 'A hearty welcome to the University of Maryland, College Park. Congratulatio

In [43]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI
from langchain.sql_database import SQLDatabase


db = SQLDatabase.from_uri("sqlite:///Chinook_Sqlite.sqlite")
#print(db.get_usable_table_names())

llm =  ChatOpenAI()

toolkit = SQLDatabaseToolkit(db=db, llm=llm)
toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000216D8F77B30>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000216D8F77B30>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000216D8F77B30>),
 QuerySQLCheckerTool(description='Use this tool to 

In [ ]:
from langchain.prompts import ChatPromptTemplate,HumanMessagePromptTemplate,MessagesPlaceholder
from langchain.schema import SystemMessage
from langchain.tools import tool
from langchain.agents import AgentExecutor,create_tool_calling_agent

prompt = ChatPromptTemplate(
    messages=[
        SystemMessage(content=(
            """You are an AI .You should not use tools parallelly 
            You should not make your own conclusions even if it is easy.
              instead use appropriate tools 
              
              IMPORTANT Rules:
              Before generating query, make sure to get the database details like
                table names,column name before generating query
                Dont assume the query. 
                Verify the query for syntax errors
              
              """
        )),

        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)
prompt

agent = create_tool_calling_agent(llm=llm, tools=toolkit.get_tools(),prompt=prompt)

agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    max_iterations=20,
    tools=toolkit.get_tools()


)

response = agent_executor.invoke({"input": "which artist acted in maximum albums"})



> Entering new AgentExecutor chain...

Invoking: `sql_db_query` with `{'query': 'SELECT artist_name, COUNT(album_id) AS total_albums FROM albums GROUP BY artist_name ORDER BY total_albums DESC LIMIT 1'}`


Error: (sqlite3.OperationalError) no such table: albums
[SQL: SELECT artist_name, COUNT(album_id) AS total_albums FROM albums GROUP BY artist_name ORDER BY total_albums DESC LIMIT 1]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_query` with `{'query': 'SELECT artist.Name AS artist_name, COUNT(album.AlbumId) AS total_albums FROM artist JOIN album ON artist.ArtistId = album.ArtistId GROUP BY artist_name ORDER BY total_albums DESC LIMIT 1'}`


[('Iron Maiden', 21)]The artist "Iron Maiden" has acted in the maximum number of albums, which is 21.

> Finished chain.
